In [2]:
pip install nltk

     ---------------------------------------- 1.5/1.5 MB 11.9 MB/s eta 0:00:00
  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)
     ------------------------------------- 262.8/262.8 kB 16.8 MB/s eta 0:00:00

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
import re
analyser = SentimentIntensityAnalyzer()

In [5]:
data=pd.read_csv("skirt-flipkart-final-final.csv")
data.head()
data=data[['URL','ID','ITEM','STARS','NUMBER OF RATINGS','NUMBER OF REVIEWS', 'LIST OF REVIEWS']]

In [6]:
data.rename(columns = {'ID':'id','ITEM':'desc', 'LIST OF REVIEWS':'reviews','NUMBER OF REVIEWS':'num_reviews','STARS':'stars','NUMBER OF RATINGS':'num_ratings'}, inplace=True)
data.head()

,URL,id,desc,stars,num_ratings,num_reviews,reviews
0,http://flipkart.com/range-road-solid-women-a-l...,0,Women Solid A-line Multicolor Skirt,4.0,38,4,['Nice quality 👍 & good colour 😍 thank you Fli...
1,http://flipkart.com/range-road-solid-women-a-l...,1,Women Solid A-line Multicolor Skirt,4.1,19,1,"['Very Best Quality, Beautiful look wise..ladi..."
2,http://flipkart.com/montrez-solid-women-a-line...,2,Women Solid A-line Light Blue Skirt,4.1,3874,422,"['Super product. Deniem material,light blue in..."
3,http://flipkart.com/montrez-solid-women-a-line...,3,Women Solid A-line Blue Skirt,4.1,3874,422,"['Super product. Deniem material,light blue in..."
4,http://flipkart.com/fkube-printed-women-pleate...,4,Women Printed Pleated Pink Skirt,4.4,1349,189,"['Woow lovely osam😀', 'Good nice material', 'L..."


In [7]:
data.reviews = data.reviews.str.lower()
data.reviews = data.reviews.str.replace('\n','').str.replace('[\'!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]','')

C:\Users\ARNAVJ~1\AppData\Local\Temp/ipykernel_17824/2430931611.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data.reviews = data.reviews.str.replace('\n','').str.replace('[\'!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]','')


In [8]:
data['vader_score'] = 1.0
for ind in data.index: 
    rev=data['reviews'][ind]
    x = rev.split()
    sum_score=0.0
    for i in x:
        score = analyser.polarity_scores(i)
        sum_score=sum_score+score['compound']
    data['vader_score'][ind] =sum_score

C:\Users\ARNAVJ~1\AppData\Local\Temp/ipykernel_17824/4164345182.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['vader_score'][ind] =sum_score


In [9]:
total_star_givers = data['num_ratings'].sum()     
total_review_givers = data['num_reviews'].sum()

In [10]:
data['final_score'] = ((data['stars']*data['num_ratings'])/total_star_givers)+((data['vader_score']*data['num_reviews'])/total_review_givers)

In [11]:
data.head()

,URL,id,desc,stars,num_ratings,num_reviews,reviews,vader_score,final_score
0,http://flipkart.com/range-road-solid-women-a-l...,0,Women Solid A-line Multicolor Skirt,4.0,38,4,nice quality 👍 good colour 😍 thank you flipka...,2.5438,0.000910
1,http://flipkart.com/range-road-solid-women-a-l...,1,Women Solid A-line Multicolor Skirt,4.1,19,1,very best quality beautiful look wiseladies go...,1.2363,0.000334
2,http://flipkart.com/montrez-solid-women-a-line...,2,Women Solid A-line Light Blue Skirt,4.1,3874,422,super product deniem materiallight blue in clo...,4.3497,0.120878
3,http://flipkart.com/montrez-solid-women-a-line...,3,Women Solid A-line Blue Skirt,4.1,3874,422,super product deniem materiallight blue in clo...,4.3497,0.120878
4,http://flipkart.com/fkube-printed-women-pleate...,4,Women Printed Pleated Pink Skirt,4.4,1349,189,woow lovely osam😀 good nice material loved the...,2.8299,0.040132


In [12]:
data.to_csv('skirts_csv_final.csv')

In [14]:
import nltk
import re
nltk.download('punkt')

dict_bigram={}
item_bigram={}
# word_data = "The best performance can bring in sky high success."
for ind in data.index:
    d=data['desc'][ind][6:-5]
    d= re.sub(r'[^\w\s]', '', d) 
    nltk_tokens = nltk.word_tokenize(d)
    arr=list(nltk.bigrams(nltk_tokens))
    for i in arr:
        if i in dict_bigram:
            dict_bigram[i]=dict_bigram[i]+1
            item_bigram[" ".join(i)].append(ind)
        else:
            dict_bigram[i]=1
            item_bigram[" ".join(i)]=[]
            item_bigram[" ".join(i)].append(ind)
print(item_bigram) 


[nltk_data] Downloading package punkt to C:\Users\arnav
[nltk_data]     jain\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


{'Solid Aline': [0, 1, 2, 3, 5, 7, 8, 10, 11, 16, 24, 25, 31, 52, 54, 62, 69, 77, 81, 98, 99, 104, 113, 128, 130, 131, 135, 139, 143, 147, 151, 172, 178, 179, 191, 193, 198, 199, 213, 215, 225, 243, 259, 261, 266, 276, 278, 281, 291, 300, 310, 321], 'Aline Multicolor': [0, 1, 7, 10, 11, 16, 24, 128, 131, 135, 139, 143, 147, 151, 193, 215, 288], 'Aline Light': [2, 278], 'Light Blue': [2, 76, 146, 166, 167, 278, 279], 'Aline Blue': [3, 5, 99, 178, 190, 198, 212, 259, 266, 283, 300], 'Printed Pleated': [4, 314], 'Pleated Pink': [4, 22, 50, 71, 157, 227], 'Solid Straight': [6, 9, 33, 76, 80, 89, 114, 152, 164, 166, 269, 299], 'Straight Dark': [6, 9, 33, 89, 145, 299], 'Dark Blue': [6, 9, 25, 33, 49, 62, 89, 94, 98, 107, 145, 161, 169, 180, 200, 209, 225, 246, 277, 281, 296, 299, 310, 313], 'Aline Black': [8, 31, 41, 54, 66, 77, 92, 104, 113, 130, 172, 183, 191, 204, 213, 276, 291, 304, 321], 'Solid Regular': [12, 30, 42, 74, 101, 108, 110, 112, 124, 134, 141, 142, 167, 238, 252, 257, 277, 

In [15]:
dict_bigram_sorted= sorted(dict_bigram, key=dict_bigram.get, reverse=True)
print(dict_bigram_sorted)

[('Solid', 'Aline'), ('Solid', 'Flared'), ('Solid', 'Pencil'), ('Solid', 'Pleated'), ('Dark', 'Blue'), ('Solid', 'Regular'), ('Floral', 'Print'), ('Aline', 'Black'), ('Self', 'Design'), ('Printed', 'Flared'), ('Aline', 'Multicolor'), ('Pencil', 'Black'), ('Flared', 'Black'), ('Solid', 'Straight'), ('Aline', 'Blue'), ('Regular', 'Black'), ('Woven', 'Gathered'), ('Gathered', 'Light'), ('Light', 'Green'), ('Flared', 'Maroon'), ('Design', 'Gathered'), ('Gathered', 'Green'), ('Aline', 'Dark'), ('Flared', 'Green'), ('Printed', 'Aline'), ('Regular', 'Blue'), ('Light', 'Blue'), ('Printed', 'Wrap'), ('Wrap', 'Around'), ('Pleated', 'Black'), ('Flared', 'Blue'), ('Pleated', 'Pink'), ('Straight', 'Dark'), ('Pleated', 'Green'), ('Print', 'Aline'), ('Print', 'Flared'), ('Pencil', 'Blue'), ('Flared', 'Pink'), ('Polka', 'Print'), ('Solid', 'Skorts'), ('Washed', 'Aline'), ('Pencil', 'Dark'), ('Print', 'Pleated'), ('Straight', 'Black'), ('Aline', 'Brown'), ('Straight', 'Blue'), ('Aline', 'White'), ('Des

In [16]:
row=dict_bigram_sorted[0:5]+dict_bigram_sorted[-5:]
print(row)

[('Solid', 'Aline'), ('Solid', 'Flared'), ('Solid', 'Pencil'), ('Solid', 'Pleated'), ('Dark', 'Blue'), ('Striped', 'Skorts'), ('Woven', 'Flared'), ('Flared', 'Purple'), ('Pleated', 'White'), ('Pleated', 'Gold')]


In [17]:
res = [] 
for el in row: 
    sub=" ".join(el)
    res.append(list(sub.split("*"))) 
print(res)

[['Solid Aline'], ['Solid Flared'], ['Solid Pencil'], ['Solid Pleated'], ['Dark Blue'], ['Striped Skorts'], ['Woven Flared'], ['Flared Purple'], ['Pleated White'], ['Pleated Gold']]


In [18]:
import csv
with open('skirt_top_bottom.csv', 'w', newline='') as file:
    res = [] 
    for el in row: 
        sub=" ".join(el)
        res.append(list(sub.split("*"))) 
    for i in range(len(res)):
        res[i]=res[i]+item_bigram[res[i][0]]
    print(res)
    writer = csv.writer(file)
    writer.writerows(res)

[['Solid Aline', 0, 1, 2, 3, 5, 7, 8, 10, 11, 16, 24, 25, 31, 52, 54, 62, 69, 77, 81, 98, 99, 104, 113, 128, 130, 131, 135, 139, 143, 147, 151, 172, 178, 179, 191, 193, 198, 199, 213, 215, 225, 243, 259, 261, 266, 276, 278, 281, 291, 300, 310, 321], ['Solid Flared', 20, 32, 36, 37, 43, 56, 57, 60, 61, 64, 65, 67, 68, 79, 87, 88, 93, 109, 120, 126, 132, 140, 144, 156, 168, 218, 219, 221, 223, 229, 232, 235, 236, 253, 254, 263, 268], ['Solid Pencil', 14, 15, 47, 58, 83, 107, 111, 150, 169, 174, 180, 181, 184, 195, 200, 201, 208, 209, 216, 222, 239, 240, 265, 284, 290, 307, 317, 320], ['Solid Pleated', 17, 18, 22, 23, 50, 63, 70, 78, 95, 136, 146, 158, 160, 170, 175, 196, 203, 217, 234, 244, 255, 274, 285, 296, 298, 315, 316, 319], ['Dark Blue', 6, 9, 25, 33, 49, 62, 89, 94, 98, 107, 145, 161, 169, 180, 200, 209, 225, 246, 277, 281, 296, 299, 310, 313], ['Striped Skorts', 301], ['Woven Flared', 311], ['Flared Purple', 311], ['Pleated White', 314], ['Pleated Gold', 315]]
